In [5]:
import pandas as pd
import os
import sys
import logging
import pandas as pd
from google.cloud import bigquery
from hashlib import md5
from typing import List

In [64]:
#defined project, dataset and eventual table name
PROJECT_NAME = "team-week-10"
DATASET_NAME = "nfl_sports_betting"
TABLE_NAME = "lk_stadiums"

#establish schema 

TABLE_METADATA = {
    "stadiums": {
        "table_name": "dim_stadiums",
        "schema": [
            bigquery.SchemaField("stadium_name", "string", mode="REQUIRED"),
            bigquery.SchemaField("stadium_location", "string", mode="NULLABLE"),
            bigquery.SchemaField("stadium_open", "float64", mode="NULLABLE"),
            bigquery.SchemaField("stadium_close", "float64", mode="NULLABLE"),
            bigquery.SchemaField("stadium_type", "string", mode="NULLABLE"),
            bigquery.SchemaField("zipcode", "string", mode="NULLABLE"),
            bigquery.SchemaField("weather_type", "string", mode="NULLABLE"),
            bigquery.SchemaField("capacity", "string", mode="NULLABLE"),
            bigquery.SchemaField("surface", "string", mode="NULLABLE"),
            bigquery.SchemaField("elevation", "float64", mode="NULLABLE"),
        ]
    }

}

logging.basicConfig(
    format='[%(levelname)-5s][%(asctime)s][%(module)s:%(lineno)04d] : %(message)s',
    level=logging.INFO,
    stream=sys.stdout
)
logger: logging.Logger = logging.getLogger("root")
logger.setLevel(logging.DEBUG)

In [24]:
#read the csv into a pandas dataframe
df = pd.read_csv("./data/nfl_stadiums.csv", header=0, encoding="unicode_escape")
display(df.head(2))


,stadium_name,stadium_location,stadium_open,stadium_close,stadium_type,stadium_address,stadium_weather_station_zipcode,stadium_weather_type,stadium_capacity,stadium_surface,stadium_weather_station,stadium_weather_station_name,stadium_latitude,stadium_longitude,stadium_azimuthangle,stadium_elevation
0,Acrisure Stadium,"Pittsburgh, PA",2001.0,NaN,outdoor,"100 Art Rooney Ave, Pittsburgh, PA 15212",15212,cold,"65,500",Grass,USW00094823,"PITTSBURGH ASOS, PA US",40.484600,-80.214400,NaN,366.7
1,Alamo Dome,"San Antonio, TX",NaN,NaN,indoor,"100 Montana St, San Antonio, TX 78203",78203,indoor,72000,FieldTurf,NaN,NaN,29.416944,-98.478889,0.0,NaN


In [55]:
stadiums_df = df.drop(columns= [
    "stadium_weather_station",
    "stadium_weather_station_name",
    "stadium_latitude",
    "stadium_longitude",
    "stadium_azimuthangle"
])

display(stadiums_df.info())
display(stadiums_df.head(2))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 11 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   stadium_name                     120 non-null    object 
 1   stadium_location                 119 non-null    object 
 2   stadium_open                     90 non-null     float64
 3   stadium_close                    41 non-null     float64
 4   stadium_type                     109 non-null    object 
 5   stadium_address                  102 non-null    object 
 6   stadium_weather_station_zipcode  101 non-null    object 
 7   stadium_weather_type             117 non-null    object 
 8   stadium_capacity                 54 non-null     object 
 9   stadium_surface                  68 non-null     object 
 10  stadium_elevation                65 non-null     float64
dtypes: float64(3), object(8)
memory usage: 10.4+ KB


None

,stadium_name,stadium_location,stadium_open,stadium_close,stadium_type,stadium_address,stadium_weather_station_zipcode,stadium_weather_type,stadium_capacity,stadium_surface,stadium_elevation
0,Acrisure Stadium,"Pittsburgh, PA",2001.0,NaN,outdoor,"100 Art Rooney Ave, Pittsburgh, PA 15212",15212,cold,"65,500",Grass,366.7
1,Alamo Dome,"San Antonio, TX",NaN,NaN,indoor,"100 Montana St, San Antonio, TX 78203",78203,indoor,72000,FieldTurf,NaN


In [60]:
#stadiums_df = stadiums_df["elevation"].astype("float")

#rename and shorten certain column names
stadiums_df = stadiums_df.rename(columns= {
    "stadium_address": "address", 
    "stadium_weather_station_zipcode": "zipcode", 
    "stadium_weather_type": "weather_type", 
    "stadium_capacity": "capacity", 
    "stadium_surface": "surface", 
    "stadium_elevation": "elevation"})

stadiums_df.info()
display(stadiums_df.head(2))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   stadium_name      120 non-null    object 
 1   stadium_location  119 non-null    object 
 2   stadium_open      90 non-null     float64
 3   stadium_close     41 non-null     float64
 4   stadium_type      109 non-null    object 
 5   address           102 non-null    object 
 6   zipcode           101 non-null    object 
 7   weather_type      117 non-null    object 
 8   capacity          54 non-null     object 
 9   surface           68 non-null     object 
 10  elevation         65 non-null     float64
dtypes: float64(3), object(8)
memory usage: 10.4+ KB


,stadium_name,stadium_location,stadium_open,stadium_close,stadium_type,address,zipcode,weather_type,capacity,surface,elevation
0,Acrisure Stadium,"Pittsburgh, PA",2001.0,NaN,outdoor,"100 Art Rooney Ave, Pittsburgh, PA 15212",15212,cold,"65,500",Grass,366.7
1,Alamo Dome,"San Antonio, TX",NaN,NaN,indoor,"100 Montana St, San Antonio, TX 78203",78203,indoor,72000,FieldTurf,NaN


In [65]:
stadiums_table_name = f"{PROJECT_NAME}.{DATASET_NAME}.{TABLE_METADATA['stadiums']['table_name']}"
stadiums_schema = TABLE_METADATA['stadiums']['schema']
client = bigquery.Client()

def load_table(
    df: pd.DataFrame, 
    client: bigquery.Client, 
    table_name: str, 
    schema: List[bigquery.SchemaField], 
    create_disposition: str = 'CREATE_IF_NEEDED', 
    write_disposition: str = 'WRITE_TRUNCATE'
    ) -> None:
    """load dataframe into bigquery table

    Args:
        df (pd.DataFrame): dataframe to load
        client (bigquery.Client): bigquery client
        table_name (str): full table name including project and dataset id
        schema (List[bigquery.SchemaField]): table schema with data types
        create_disposition (str, optional): create table disposition. Defaults to 'CREATE_IF_NEEDED'.
        write_disposition (str, optional): overwrite table disposition. Defaults to 'WRITE_TRUNCATE'.
    """

    # test table name to be full table name including project and dataset name. It must contain to dots
    assert len(table_name.split('.')) == 3, f"Table name must be a full bigquery table name including project and dataset id: '{table_name}'"


    job_config = bigquery.LoadJobConfig(
        create_disposition=create_disposition,
        write_disposition=write_disposition,
        schema=schema
    )
    logger.info(f"loading table: '{table_name}'")
    job = client.load_table_from_dataframe(df, destination=table_name, job_config=job_config)
    job.result() 

    # get the resulting table
    table = client.get_table(table_name)
    logger.info(f"loaded {table.num_rows} rows into {table.full_table_id}")

load_table(stadiums_df, client, stadiums_table_name, stadiums_schema)

[DEBUG][2024-01-13 19:55:33,949][_default:0255] : Checking /Users/kairo/.creds/dsa-deb-sa.json for explicit credentials as part of auth process...
[DEBUG][2024-01-13 19:55:33,952][_default:0255] : Checking /Users/kairo/.creds/dsa-deb-sa.json for explicit credentials as part of auth process...
[INFO ][2024-01-13 19:55:33,955][3792743445:0033] : loading table: 'team-week-10.nfl_sports_betting.dim_stadiums'
[DEBUG][2024-01-13 19:55:33,960][retry:0282] : Converted retries value: 3 -> Retry(total=3, connect=None, read=None, redirect=None, status=None)
[DEBUG][2024-01-13 19:55:34,002][requests:0185] : Making request: POST https://oauth2.googleapis.com/token
[DEBUG][2024-01-13 19:55:34,058][connectionpool:1052] : Starting new HTTPS connection (1): oauth2.googleapis.com:443
[DEBUG][2024-01-13 19:55:34,169][connectionpool:0546] : https://oauth2.googleapis.com:443 "POST /token HTTP/1.1" 200 None
[DEBUG][2024-01-13 19:55:34,262][connectionpool:1052] : Starting new HTTPS connection (1): bigquery.g